In [1]:
import numpy as np, pandas as pd, pystan as ps, numpy.random as npr, matplotlib.pyplot as plt, h5py
%matplotlib inline 
from time import time
from pylab import plot, show, legend
from scipy.stats import pearsonr, spearmanr, norm, invgamma

In [2]:
sm_HS = ps.StanModel(file="horseshoe_logistic.stan") 

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5218e2900be47b3d461b8ba25a5dd4f5 NOW.


In [3]:
data = h5py.File("GZZ_data.jld", "r")
X = data["X"].value
y = data["y"].value
ξ_true = data["xi_true"].value
d, Nobs = np.shape(X.transpose())

data = dict(N=Nobs, d=d, y=y.astype(int), X=X)

In [4]:
start = time()
fit_HS = sm_HS.sampling(data=data, thin=1, n_jobs=4, init="random", iter=5_000, algorithm="HMC")
print(round((time()-start)/60, 2), "mins to run")
print(fit_HS);

130.51 mins to run
Inference for Stan model: anon_model_5218e2900be47b3d461b8ba25a5dd4f5.
4 chains, each with iter=5000; warmup=2500; thin=1; 
post-warmup draws per chain=2500, total post-warmup draws=10000.

              mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
xi[1]        -0.09  6.5e-3   0.31  -0.74   -0.3  -0.09   0.11   0.52   2310    1.0
xi[2]         -0.1  7.4e-3   0.55  -1.47  -0.25  -0.01   0.12   0.93   5399    1.0
xi[3]         0.04    0.02   0.99  -1.97  -0.19 5.1e-3   0.24   2.14   3757    1.0
xi[4]        -1.03    0.05   2.48  -8.27  -1.07  -0.15   0.04   0.83   2035    1.0
xi[5]         0.06  6.6e-3   0.42  -0.74  -0.11   0.01   0.19   1.07   4133    1.0
xi[6]        -0.11  8.1e-3   0.49  -1.36  -0.27  -0.03   0.09   0.79   3580    1.0
xi[7]         1.04    0.08   2.36  -0.67  -0.01    0.2   1.17   7.42    903    1.0
xi[8]          0.6    0.02   1.04  -0.62-6.1e-3   0.23   0.95   3.47   2620    1.0
xi[9]         0.34    0.01    0.8  -0.66  -0

In [6]:
trace = fit_HS.extract()
xi_samples = trace["xi"]

In [7]:
np.shape(xi_samples)

(10000, 50)

In [8]:
cover = np.zeros(d)
ci = np.zeros((d,2))
for i in range(d) :
    ci[i,:] = np.percentile(xi_samples[:,i], q=[5, 95])
    cover[i] = (ci[i,0]<ξ_true[i])&(ξ_true[i]<ci[i,1])

In [9]:
100*np.mean(cover)

96.0